 **Deep Learning Approach with DestilBert**

In [ ]:
# Google Colab Specific
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NLP_Ass_GC/project-group16/tuwnlpie/milestone2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP_Ass_GC/project-group16/tuwnlpie/milestone2


In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/e4/ae/2ad8820045b6631965750435f28583e80905b8273d57cf026163b51323ee/torch-2.1.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB 2.6 MB/s eta 0:01:16
   ---------------------------------------- 0.6/192.3 MB 8.9 MB/s eta 0:00:22
   ---------------------------------------- 2.0/192.3 MB 17.9 MB/s eta 0:00:11
    --------------------------------------- 3.4/192.3 MB 21.8 MB/s eta 0:00:09
   - -------------------------------------- 5.4/192.3 MB 26.2 MB/s eta 0:00:08
   - -------------------------------------- 6.6/192.3 MB 28.2 MB/s eta 0:00:07
   - -------------------------------------- 8.0/192.3 MB 28.4 MB/s eta 0:00:07
   - -------------------------------------- 9.1/192.3 MB 27.8 MB/s eta 0:00:07
   -- ------------------------------------- 10.4/192.3 MB 31.2 MB/s eta 0:00:06
   -- ---------

In [3]:
import torch
import pandas as pd
import numpy as np
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import torch.optim as optim

**Importing Data**

In [4]:
from utils import get_data
import utils
from sklearn import metrics
from sklearn.pipeline import Pipeline

X_train, y_train, X_test, y_test = get_data()

loaded df from ../../data/dreaddit-train.csv with 115 columns.
number of na values in the whole df: 0
loaded df from ../../data/dreaddit-test.csv with 115 columns.
number of na values in the whole df: 0


In [5]:
# Taking 70% of the whole dataset as it is very slow to train
percentage_to_keep = 0.7
num_rows_to_keep_test = int(len(X_test) * percentage_to_keep)
num_rows_to_keep_train = int(len(X_train) * percentage_to_keep)

y_test = y_test[:num_rows_to_keep_test]
X_test = X_test[:num_rows_to_keep_test]
y_train = y_train[:num_rows_to_keep_train]
X_train = X_train[:num_rows_to_keep_train]

**Model**

We decided to implement the  "distilbert-base-uncased-finetuned-sst-2-english" as our DL baseline model. It is a sentiment analysis model based on DistilBERT, fine-tuned for English using the SST-2 dataset.
Since we thought it could be useful for tasks related to understanding sentiment in English texts, we decided to employ it for our current tasks. The training and validation was performed with only tokens of around 70% of the data, in hope of expediting the training process. Also, only the weights of last layer were updated, all other layers were left frozen.

In [6]:
from transformers import AutoModel, AutoTokenizer


model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModel.from_pretrained(model_name)
for p in model.base_model.parameters():
    p.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\reema\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\reema\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


**Training**

In [7]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Tokenizing and creating attention masks for training data
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
X_train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
attention_mask_train = X_train_tokens['attention_mask']

# Tokenizing and creating attention masks for testing data
X_test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt')
attention_mask_test = X_test_tokens['attention_mask']

# Defining batch size
batch_size = 64

# Creating DataLoader for training data
train_dataset = TensorDataset(X_train_tokens['input_ids'], attention_mask_train, torch.tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Creating DataLoader for testing data
test_dataset = TensorDataset(X_test_tokens['input_ids'], attention_mask_test, torch.tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader))

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs
num_epochs = 3
best_val_loss = float('inf')
precision_per_epoch = []
recall_per_epoch = []
f1_per_epoch = []
accuracy_per_epoch = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    i = 0
    total_loss = 0.0
    # Training loop
    for batch in train_dataloader:
        inputs, attention_mask, labels = batch  # Include attention_mask
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask)  # Pass attention_mask to the model
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        if (i % 2 == 0):
            print(f'Batch {i+1} in epoch {epoch+1} finished training.')
        total_loss += loss.item()
        i = i + 1

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Training for epoch {epoch+1} finished.')

    # Validation
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    for batch in test_dataloader:
        inputs, attention_mask, labels = batch  # Include attention_mask
        with torch.no_grad():
            outputs = model(inputs, attention_mask=attention_mask)  # Pass attention_mask to the model
        loss = criterion(outputs.logits, labels)
        val_loss += loss.item()
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().numpy() + 1
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(test_dataloader)
    # reversed because no-stress = 0, and stress = 1
    y_pred_binary = [0 if label == 1 else 1 for label in all_preds]
    precision = precision_score(all_labels, y_pred_binary, zero_division=1)
    precision_per_epoch.append(precision)
    accuracy = accuracy_score(all_labels, y_pred_binary)
    accuracy_per_epoch.append(accuracy)
    recall = recall_score(all_labels, y_pred_binary, zero_division=1)
    recall_per_epoch.append(recall)
    f1 = f1_score(all_labels, y_pred_binary)
    f1_per_epoch.append(f1)

    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.3f}, Training Loss: {avg_train_loss:.3f}')
    print(f'Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}, Accuracy Score: {accuracy:.3f}')
    # Check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
    else:
        print("Validation loss is not improving.")
        break

# Ending Training
end_time = time.time()

Batch 1 in epoch 1 finished training.
Batch 3 in epoch 1 finished training.
Batch 5 in epoch 1 finished training.
Batch 7 in epoch 1 finished training.
Batch 9 in epoch 1 finished training.
Batch 11 in epoch 1 finished training.
Batch 13 in epoch 1 finished training.
Batch 15 in epoch 1 finished training.
Batch 17 in epoch 1 finished training.
Batch 19 in epoch 1 finished training.


KeyboardInterrupt: 

In [18]:
# Saving and downloading the model 
# from google.colab import files
# torch.save(model.state_dict(), 'distilbert_initial_model_MS2.pth')
# files.download('distilbert_initial_model_MS2.pth')


import torch

# Save your model
torch.save(model.state_dict(), 'saved_models/distilbert_initial_model_MS2.pth')


In [19]:
training_time = (end_time - start_time)/60
print(f"Time [in min] it took to train and evaluate the model: {training_time:.1f}")

NameError: name 'end_time' is not defined

**Analysis**

In [20]:
# Preparing the data for the analysis as the test dataset were not shuffled
labels_last_epoch = all_labels
predicitions_last_epoch = y_pred_binary
mismatched_indices = [i for i, (true, pred) in enumerate(zip(labels_last_epoch, predicitions_last_epoch)) if true != pred]
mismatched_indices[0:10]

NameError: name 'all_labels' is not defined

**Confussion Matrix**

In [21]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(all_labels, y_pred_binary)
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

NameError: name 'all_labels' is not defined

This analysis suggests that the model's performance is characterized by a precision of 0.644, indicating the proportion of correctly identified positive instances among all instances labeled as positive. The recall is 0.934, signifying the proportion of actual positive instances that were correctly identified by the model. The F1 score, which balances precision and recall, is 0.762. The accuracy score, representing the overall correctness of the model's predictions, is 0.698. Overall, the model demonstrates a high recall, indicating its effectiveness in capturing positive instances, but there is room for improvement in precision. It can also be concluded that there more instances were incorrectly indentified as positive compared to instnaces that were incorrectly indentified as negative.

**Individual Cases**

In [ ]:
test_dataset = pd.read_csv('../../data/dreaddit-test.csv', index_col='id')
test_dataset.drop(test_dataset[test_dataset.text.str.strip().str.len() < 50].index, inplace=True)
test_conf = test_dataset['confidence'].to_list()

In [ ]:
# Checking the mismatch
mismatched_text = [X_test[i] for i in mismatched_indices]
mismatched_conf = [test_conf[i] for i in mismatched_indices]
real_y = [y_test[i] for i in mismatched_indices]

data = {
    'idx': mismatchdataed_indices,
    'text': mismatched_text,
    'confidence': mismatched_conf,
    'real_Y': real_y
}
df_mismatched = pd.DataFrame(data)
df_mismatched.to_csv('mismatches.csv', index=False)

In [ ]:
df_mismatched.head(5)

idx                                               text  confidence  real_Y
0    0  Its like that, if you want or not.“ ME: I have...         0.8       0
1    4  I haven’t said anything to him yet because I’m...         0.8       0
2   15  Despite being younger than him, I have had mor...         0.8       0
3   18  If I go to an interview for example, I'll know...         0.6       0
4   19  Like sleep would never be a simple thing for m...         0.0       0

Analysis is continued in Analysis_DLmethod_ML2.ipynb notebook